In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("files")

In [ ]:
import csv


def save_to_csv(filename, data):
    filename = "files/" + filename + ".csv"
    print(f"Saving data to {filename}")
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)
    print(f"Data saved to {filename}")


def read_from_csv(filename, variable_type=np.float64):
    filename = "files/" + filename + ".csv"
    with open(filename, newline='') as file:
        reader = csv.reader(file)
        data = list(reader)
        for arr in range(len(data)):
            data[arr] = np.array(data[arr], dtype=variable_type)
    return data

In [ ]:
def get_random_sign():
    return np.random.choice([-1, 1])


def get_x(n, variable_type=np.float64):
    x_ = np.zeros(n, dtype=variable_type)
    for i in range(n):
        x_[i] = variable_type(get_random_sign())
    return x_


def get_b(A, x, n, variable_type=np.float64):
    b = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(n):
            b[i] += A[i][j] * x[j]
    return b

In [ ]:
k = np.float64(6)
m = np.float64(4)
one = np.float64(1)


def get_A(size):
    global k, m, one
    A = np.zeros((size, size), dtype=np.float64)
    for i in range(size):
        for j in range(size):
            i_ = np.float64(i)
            j_ = np.float64(j)

            if i == j:
                A[i][j] = k
            elif j > i:
                A[i][j] = (-one) ** (j_ + one) * m / (j_ + one)
            elif j + 1 == i:
                A[i][j] = m / (i_ + one)
            else:
                A[i][j] = np.float64(0)
    return A


def get_D(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        D[i][i] = k
    return D


def get_D_inverted(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    k_ = np.float64(k)
    for i in range(n):
        D[i][i] = one / k_
    return D


def get_negative_D_inverted(n):
    global k, m, one
    D = np.zeros((n, n), dtype=np.float64)
    k_ = np.float64(k)
    minus_one = np.float64(-1)
    for i in range(n):
        D[i][i] = minus_one / k_
    return D


def get_L(n):
    global k, m, one
    L = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        for j in range(n):
            i_ = np.float64(i)
            j_ = np.float64(j)
            if i == j + 1:
                L[i][j] = m / (i_ + one)
    return L


def get_U(n):
    global k, m, one
    U = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        for j in range(n):
            i_ = np.float64(i)
            j_ = np.float64(j)
            if j > i:
                U[i][j] = (-one) ** (j_ + one) * m / (j_ + one)
    return U


def get_M(n):
    U = get_U(n)
    L = get_L(n)
    negative_D_inverted = get_negative_D_inverted(n)
    M = np.dot(negative_D_inverted, L + U)
    return M


In [ ]:
import time


def multiply_matrices(A, B):
    if not isinstance(B[0], np.ndarray):
        result = np.zeros(len(A), dtype=np.float64)
        for i in range(len(A)):
            for j in range(len(B)):
                result[i] += A[i][j] * B[j]
        return result
    else:
        result = np.zeros((len(A), len(B[0])), dtype=np.float64)
        for i in range(len(A)):
            for j in range(len(B[0])):
                for k in range(len(B)):
                    result[i][j] += A[i][k] * B[k][j]
        return result


def norm(vector):
    max_value = -np.inf
    for value in vector:
        if np.abs(value) > max_value:
            max_value = np.abs(value)
    return max_value


class Jacobi:
    def __init__(self, b, ceil_limitation, rho, stop_criterium, x_start, proper_result):
        self.n = len(b)
        self.A = get_A(self.n)
        self.b = b
        self.ceil_limitation = ceil_limitation
        self.M = get_M(self.n)
        self.N = get_D_inverted(self.n)
        self.proper_result = proper_result
        self.rho = rho
        self.stop_criterium = stop_criterium
        self.x = x_start
        if stop_criterium != "res" and stop_criterium != "diff":
            raise Exception("Wrong stop criterium")

    def residual_stop_criterium(self):
        return norm(multiply_matrices(self.A, self.x) - self.b) < self.rho

    def difference_stop_criterium(self, x_next):
        return norm(x_next - self.x) < self.rho

    def calculate(self):
        start = time.time()
        for i in range(self.ceil_limitation):
            x_next = multiply_matrices(self.M, self.x) + multiply_matrices(self.N, self.b)
            if self.stop_criterium == "res":
                if self.residual_stop_criterium():
                    return x_next, norm(x_next - self.proper_result), i + 1, time.time() - start
            elif self.stop_criterium == "diff":
                if self.difference_stop_criterium(x_next):
                    return x_next, norm(x_next - self.proper_result), i + 1, time.time() - start
            self.x = x_next
        return x_next, norm(x_next - self.proper_result), self.ceil_limitation, time.time() - start


In [ ]:
def draw_plot(domain, results, title, x_label="PLACEHOLDER LABEL", y_label="PLACEHOLDER LABEL", scale_type=None,
              filename=None, label="PLACEHOLDER LABEL", x_ticks=None):
    default_font_size = 25
    legend_font_size = default_font_size
    title_font_size = default_font_size
    tick_params_label_size = default_font_size
    x_y_label_size = default_font_size

    fig, ax = plt.subplots(figsize=(15, 7.5))
    if scale_type is not None:
        ax.set_yscale(scale_type)
    plt.scatter(x=domain, y=results, label=label)

    plt.title(title, fontsize=title_font_size)
    plt.xlabel(x_label, fontsize=x_y_label_size)
    plt.ylabel(y_label, fontsize=x_y_label_size)
    plt.legend(fontsize=legend_font_size)

    ax.tick_params(axis='both', which='major', labelsize=tick_params_label_size)
    ax.tick_params(axis='both', which='minor', labelsize=tick_params_label_size)

    if x_ticks is not None:
        plt.xticks(x_ticks)

    plt.grid(True)
    plt.savefig(f"plots/{filename}.png")
    plt.show()

**Wykonanie eksperymentów**

In [ ]:
system_sizes = range(5, 500, 15)


def generate_data(filename, cur_sizes=None):
    if os.path.exists("files/proper_results.csv"):
        if input("Do you want to overwrite files/proper_results.csv? (y/n)") != "y":
            return
    global system_sizes
    if cur_sizes is None:
        cur_sizes = system_sizes
    xes = []
    for size in cur_sizes:
        xes.append(get_x(size, np.float64))
    return save_to_csv(filename, xes)


#generate_data("proper_results")

In [ ]:
def perform_experiments(x_start, rho):
    global system_sizes
    proper_results = read_from_csv("proper_results", np.float64)
    csv_header = ["n", "kryt. rezydualne iteracje", "kryt. przyrostowe iteracje", "kryt. rezydualne czas",
                  "kryt. przyrostowe czas", "kryt. rezydualne błąd", "kryt. przyrostowe błąd"]
    csv_file = [csv_header]


    for proper_result in proper_results:
        print("performing experiment for n = " + str(len(proper_result)) + "...")
        n = len(proper_result)
        x_start_vec = np.full(n, x_start, dtype=np.float64)
        jacobi_res = Jacobi(b=get_b(get_A(n), proper_result, n),
                            ceil_limitation=10000,
                            rho=rho,
                            stop_criterium="res",
                            x_start=x_start_vec,
                            proper_result=proper_result)
        res_result, rex_error, res_iterations, res_time = jacobi_res.calculate()

        jacobi_diff = Jacobi(b=get_b(get_A(n), proper_result, n),
                             ceil_limitation=10000,
                             rho=rho,
                             stop_criterium="diff",
                             x_start=x_start_vec,
                             proper_result=proper_result)
        diff_result, diff_error, diff_iterations, diff_time = jacobi_diff.calculate()
        csv_file.append([n, res_iterations, diff_iterations, res_time, diff_time, rex_error, diff_error])
    return save_to_csv(f"result_rho_{rho}_start_{x_start}", csv_file)

In [160]:
perform_experiments(0, np.float64(0.001))

performing experiment for n = 5...
performing experiment for n = 20...
performing experiment for n = 35...
performing experiment for n = 50...
performing experiment for n = 65...
performing experiment for n = 80...
performing experiment for n = 95...
performing experiment for n = 110...
performing experiment for n = 125...
performing experiment for n = 140...
performing experiment for n = 155...
performing experiment for n = 170...
performing experiment for n = 185...
performing experiment for n = 200...
performing experiment for n = 215...
performing experiment for n = 230...
performing experiment for n = 245...
performing experiment for n = 260...
performing experiment for n = 275...
performing experiment for n = 290...
performing experiment for n = 305...
performing experiment for n = 320...
performing experiment for n = 335...
performing experiment for n = 350...
performing experiment for n = 365...
performing experiment for n = 380...
performing experiment for n = 395...
performin

In [161]:
perform_experiments(0, np.float64(0.000001))

performing experiment for n = 5...
performing experiment for n = 20...
performing experiment for n = 35...
performing experiment for n = 50...
performing experiment for n = 65...
performing experiment for n = 80...
performing experiment for n = 95...
performing experiment for n = 110...
performing experiment for n = 125...
performing experiment for n = 140...
performing experiment for n = 155...
performing experiment for n = 170...
performing experiment for n = 185...
performing experiment for n = 200...
performing experiment for n = 215...
performing experiment for n = 230...
performing experiment for n = 245...
performing experiment for n = 260...
performing experiment for n = 275...
performing experiment for n = 290...
performing experiment for n = 305...
performing experiment for n = 320...
performing experiment for n = 335...
performing experiment for n = 350...
performing experiment for n = 365...
performing experiment for n = 380...
performing experiment for n = 395...
performin

In [162]:
perform_experiments(10, np.float64(0.001))

performing experiment for n = 5...
performing experiment for n = 20...
performing experiment for n = 35...
performing experiment for n = 50...
performing experiment for n = 65...
performing experiment for n = 80...
performing experiment for n = 95...
performing experiment for n = 110...
performing experiment for n = 125...
performing experiment for n = 140...
performing experiment for n = 155...
performing experiment for n = 170...
performing experiment for n = 185...
performing experiment for n = 200...
performing experiment for n = 215...
performing experiment for n = 230...
performing experiment for n = 245...
performing experiment for n = 260...
performing experiment for n = 275...
performing experiment for n = 290...
performing experiment for n = 305...
performing experiment for n = 320...
performing experiment for n = 335...
performing experiment for n = 350...
performing experiment for n = 365...
performing experiment for n = 380...
performing experiment for n = 395...
performin

In [163]:
perform_experiments(10, np.float64(0.000001))

performing experiment for n = 5...
performing experiment for n = 20...
performing experiment for n = 35...
performing experiment for n = 50...
performing experiment for n = 65...
performing experiment for n = 80...
performing experiment for n = 95...
performing experiment for n = 110...
performing experiment for n = 125...
performing experiment for n = 140...
performing experiment for n = 155...
performing experiment for n = 170...
performing experiment for n = 185...
performing experiment for n = 200...
performing experiment for n = 215...
performing experiment for n = 230...
performing experiment for n = 245...
performing experiment for n = 260...
performing experiment for n = 275...
performing experiment for n = 290...
performing experiment for n = 305...
performing experiment for n = 320...
performing experiment for n = 335...
performing experiment for n = 350...
performing experiment for n = 365...
performing experiment for n = 380...
performing experiment for n = 395...
performin

In [164]:
perform_experiments(-10, np.float64(0.001))

performing experiment for n = 5...
performing experiment for n = 20...
performing experiment for n = 35...
performing experiment for n = 50...
performing experiment for n = 65...
performing experiment for n = 80...
performing experiment for n = 95...
performing experiment for n = 110...
performing experiment for n = 125...
performing experiment for n = 140...
performing experiment for n = 155...
performing experiment for n = 170...
performing experiment for n = 185...
performing experiment for n = 200...
performing experiment for n = 215...
performing experiment for n = 230...
performing experiment for n = 245...
performing experiment for n = 260...
performing experiment for n = 275...
performing experiment for n = 290...
performing experiment for n = 305...
performing experiment for n = 320...
performing experiment for n = 335...
performing experiment for n = 350...
performing experiment for n = 365...
performing experiment for n = 380...
performing experiment for n = 395...
performin

In [165]:
perform_experiments(-10, np.float64(0.000001))

performing experiment for n = 5...
performing experiment for n = 20...
performing experiment for n = 35...
performing experiment for n = 50...
performing experiment for n = 65...
performing experiment for n = 80...
performing experiment for n = 95...
performing experiment for n = 110...
performing experiment for n = 125...
performing experiment for n = 140...
performing experiment for n = 155...
performing experiment for n = 170...
performing experiment for n = 185...
performing experiment for n = 200...
performing experiment for n = 215...
performing experiment for n = 230...
performing experiment for n = 245...
performing experiment for n = 260...
performing experiment for n = 275...
performing experiment for n = 290...
performing experiment for n = 305...
performing experiment for n = 320...
performing experiment for n = 335...
performing experiment for n = 350...
performing experiment for n = 365...
performing experiment for n = 380...
performing experiment for n = 395...
performin

In [ ]:
def spectral_radius(matrix):
    eigenvalues = np.abs(np.linalg.eigvals(matrix))
    return np.max(eigenvalues)


'''for n in range(3, 150):
    M = get_M(n)
    print(f"n = {n}, spectral radius = {spectral_radius(M)}")'''  #%%


def spectral_radius(matrix):
    eigenvalues = np.abs(np.linalg.eigvals(matrix))
    return np.max(eigenvalues)


'''for n in range(3, 150):
    M = get_M(n)
    print(f"n = {n}, spectral radius = {spectral_radius(M)}")'''

In [ ]:
import numpy as np


def generate_random_system(num_equations, num_variables):
    A = np.random.rand(num_equations, num_variables)
    b = np.random.rand(num_equations)
    return A, b


def test_jacobi_vs_np_solve(n):
    for i in range(1000):
        _, b = generate_random_system(n, n)
        A = get_A(n)
        np_solution = np.linalg.solve(A, b)
        jacobi_solver = Jacobi(b=b, ceil_limitation=1000, rho=0.0000000001, stop_criterium="diff",
                               x_start=np.zeros(n), proper_result=np_solution)
        jacobi_solution = jacobi_solver.calculate()[0]

        if np.allclose(np_solution, jacobi_solution):
            print("The solutions match!")
        else:
            print(f"The solutions do not match. np_solution = {np_solution}, jacobi_solution = {jacobi_solution}")
            return


# Test the Jacobi class against np.linalg.solve
test_jacobi_vs_np_solve(10)